In [6]:
import numpy as np 
import pandas as pd 
import time  
import jieba 
import datetime
from pyecharts import options as opts
from pyecharts.charts import *
from pyecharts.commons.utils import JsCode

In [7]:
df1 = pd.read_excel(r'C:\Users\crz\Downloads\tianqi\北京历史天气数据.xlsx')
df2 = pd.read_excel(r'C:\Users\crz\Downloads\tianqi\广州历史天气数据.xlsx')
df3 = pd.read_excel(r'C:\Users\crz\Downloads\tianqi\上海历史天气数据.xlsx')
df4 = pd.read_excel(r'C:\Users\crz\Downloads\tianqi\深圳历史天气数据.xlsx')
data = pd.concat([df1,df2,df3,df4],sort=True)

In [8]:
data.sample(5)

城市      天气               日期 最低气温 最高气温            风向
1274  上海  多云转雷阵雨  2014-07-10 星期四   26℃  34℃  北风 南风转西风3-4级
39    广州       阴     2011-02-20    10℃  15℃      无持续风向 微风
3330  北京    多云转阴  2020-02-26 星期三   -2℃   9℃         南风 2级
832   北京    雾转阵雨  2013-04-23 星期二    9℃  17℃       微风 小于3级
194   深圳       晴     2011-07-25    28℃  33℃      无持续风向 微风

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15764 entries, 0 to 3933
Data columns (total 6 columns):
城市      15764 non-null object
天气      15764 non-null object
日期      15764 non-null object
最低气温    15764 non-null object
最高气温    15764 non-null object
风向      15764 non-null object
dtypes: object(6)
memory usage: 862.1+ KB


In [10]:
data[['日期','星期']] = data['日期'].str.split(' ',expand=True,n=1)

In [11]:
data[['最高气温','最低气温']] = data[['最高气温','最低气温']].apply(lambda x: x.str.replace('℃',''))

In [12]:
data.loc[data['天气'].str.contains('雪'),'下雪吗']='是'
data.fillna('否',inplace=True)

In [13]:
data['日期'] = pd.to_datetime(data['日期'])
data[['最高气温','最低气温']] = data[['最高气温','最低气温']].astype('int')

In [14]:
data['年份'] = data['日期'].dt.year
data['月份'] = data['日期'].dt.month
data['日'] = data['日期'].dt.day
# 预览
data.sample(5)

城市  天气         日期  最低气温  最高气温        风向    星期 下雪吗    年份  月份   日
2243  广州  小雨 2017-03-08    13    16     北风 1级  星期三    否  2017   3   8
372   广州  小雨 2012-01-20    12    17  无持续风向 微风         否  2012   1  20
2582  上海  多云 2018-02-08     0     7     北风 2级  星期四    否  2018   2   8
2250  上海  小雨 2017-03-13     6    10    西北风 1级  星期一    否  2017   3  13
2432  北京   晴 2017-09-11    16    29     南风 1级  星期一    否  2017   9  11

In [15]:
s_data = data[data['下雪吗']=='是']
s_data[(s_data['月份']>=9)].groupby('年份').first().reset_index()

年份  城市      天气         日期  最低气温  最高气温               风向    星期 下雪吗  月份   日
0  2011  北京    晴~小雪 2011-11-28    -1    10         无持续风向 微风         是  11  28
1  2012  北京  雨夹雪转多云 2012-11-05     0    10     北风 4-5级~3-4级  星期一    是  11   5
2  2013  北京      小雪 2013-12-17    -4     1          微风 小于3级  星期二    是  12  17
3  2014  北京    小雪转晴 2014-12-07    -4     5  微风~北风 小于3级~3-4级  星期日    是  12   7
4  2015  北京  小雨~雨夹雪 2015-11-05     2    10       南风~北风 3-4级  星期四    是  11   5
5  2016  北京     雨夹雪 2016-11-20    -1     5           东北风 3级  星期日    是  11  20
6  2018  上海       雪 2018-12-30     0     4            北风 3级  星期日    是  12  30
7  2019  北京       雪 2019-11-29    -5     1           东南风 2级  星期五    是  11  29

In [16]:
s_data.groupby(['城市', '年份'])['日期'].count().to_frame('下雪天数').reset_index()

城市    年份  下雪天数
0   上海  2011     2
1   上海  2012     6
2   上海  2013     9
3   上海  2014     3
4   上海  2015     2
5   上海  2016     4
6   上海  2017     1
7   上海  2018     4
8   上海  2019     1
9   上海  2020     1
10  北京  2011    11
11  北京  2012    13
12  北京  2013    15
13  北京  2014     6
14  北京  2015    15
15  北京  2016     6
16  北京  2017     6
17  北京  2018     2
18  北京  2019     2
19  北京  2020     6
20  北京  2021     1
21  深圳  2011     1

In [17]:
s_data[s_data['城市'] == '深圳']

城市     天气         日期  最低气温  最高气温        风向 星期 下雪吗    年份  月份   日
33  深圳  小雪~小雨 2011-02-14     7    13  无持续风向 微风      是  2011   2  14

In [18]:
data_bj = data[(data['年份'] == 2021) & (data['城市'] == '北京')]
data_bj = data_bj.groupby(['月份','天气'], as_index=False)['日期'].count()
data_bj

月份     天气  日期
0    1     多云   8
1    1      晴  23
2    2     多云   4
3    2     小雪   1
4    2      晴  18
5    2      阴   2
6    2      霾   3
7    3     多云   7
8    3      晴  15
9    3      阴   6
10   3      霾   3
11   4     多云   8
12   4      晴  16
13   4      阴   6
14   5     多云  16
15   5  多云转小雨   3
16   5     小雨   2
17   5   小雨转阴   1
18   5      晴   4
19   5      阴   4
20   5      霾   1
21   6     多云   7
22   6     小雨   2
23   6  小雨到中雨   1
24   6   小雨转阴   1
25   6      晴  11
26   6      阴   8
27   7     多云   8
28   7  多云转小雨   2
29   7     小雨   1
30   7  小雨到中雨   2
31   7      晴   7
32   7      阴  11
33   8     中雨   1
34   8     多云  14
35   8     小雨   2
36   8  小雨转多云   1
37   8      晴  10
38   8      阴   3
39   9     多云  15
40   9     小雨   2
41   9  小雨到中雨   1
42   9      晴   6
43   9      阴   5
44   9   阴到小雨   1
45  10     多云  11
46  10     小雨   2
47  10      晴  15
48  10      阴   3

In [27]:
data_pivot =  pd.pivot(data_bj,values='日期',index ='天气',columns='月份')
data_pivot = data_pivot.astype('float')

data_pivot.sort_index(ascending=False,inplace=True)

data_pivot

TypeError: pivot_simple() got multiple values for argument 'index'